# Chapter 11 [통계적 가설검정]

**통계적 가설검정** : 모집단의 모수에 관한 가설을 세우고 표본의 정보를 이용해 그 가설을 검증하는 기법

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

%precision 3
np.random.seed(1111)

In [2]:
df = pd.read_csv('../../../Source/누구나 파이썬 통계분석/source/python_stat_sample-master/data/ch11_potato.csv')
sample = np.array(df['무게'])

sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [3]:
s_mean = np.mean(sample)

s_mean

128.4507142857143

## 11.1 통계적 가설검정

**통계적 가설검정(statistical hypothesis testing)**<br>
: 모집단의 모수게 관하여 두 가지 가설을 세우고, 표본으로부터 계산되는 통계량을 이용하여 어느 가설이 옳은지 판단하는 통계적인 방법

### 11.1.1 통계적 가설검정의 기본

In [4]:
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95)

128.68118313069039

***대립가설(alternative hypothesis, $H_1$)** : 주장하고 싶은 가설로 '차이가 있다.'나 '효과가 있다.'라는 내용<br>
***귀무가설(null hypothesis, $H_0$)** : '차이가 없다.'나 '효과가 없다.'
<br><br>
기각역(rejenction region) : 귀무가설이 기각되는 구간<br>
채택역(accoptance region) : 귀무가설이 채택되는 구간

***유의수준(level of significance)** : 기각역에 들어가는 확률<br>
임곗값(critical value) : 경계선상의 값<br>
***검정통계량(test statistic)** : 검정에 사용되는 통계량
<br><br>
****$p$값($p-value$)*** : 검정통계량보다 왼쪽에 있는 영역의 면적<br>
→ <span style="color:red">**$p$값이 유의수준보다 작을 때는 귀무가설을 기각하고, 그렇지 않을 때는 귀무가설을 채택**</span>

In [5]:
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [6]:
rv = stats.norm()
rv.isf(0.95)

-1.6448536269514722

In [7]:
rv.cdf(z)

0.026661319523126635

$p$값이 0.027로 유의수준 0.05보다 작은 값이므로, 귀무가설은 기각

### 11.1.2 단측검정과 양측검정

In [8]:
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [9]:
rv = stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

양측검정에서는 귀무가설이 기각되지 않는다.<br>
<br>
양측검정과 단측검정을 비교하면, **단측검정 쪽이 귀무가설을 기각하기 쉽다.**

<br>양측검정의 p값을 구할 때는 상단과 하단의 양쪽 면적을 고려할 필요가 있으므로 **누적밀도함수의 값을 2배**

In [10]:
rv.cdf(z) * 2

0.05332263904625327

### 11.1.3 가설검정의 두 가지 오류

****가설검정의 오류***<br>
- **제1종 오류** : 귀무가설이 옳을 때, 귀무가설을 기각하는 오류<br>
- **제2종 오류** : 대립가설이 옳을 때, 귀무가설을 채택하는 오류

In [11]:
# 제1종 오류
rv = stats.norm(130, 3)

In [12]:
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
cnt / n_samples

0.053

In [13]:
# 제2종 오류
rv = stats.norm(128, 3)

In [14]:
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
        
cnt / n_samples

0.197

## 11.2 기본적인 가설검정

### 11.2.1 정규분포의 모평균에 대한 검정 : 모분산을 알고 있는 경우

In [15]:
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)

    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'p값은 {p:.3f}')

In [16]:
pmean_test(sample, 130, 9)

귀무가설을 채택
p값은 0.053


### 11.2.2 정규분포의 모분산에 대한 검정

- 검정통계량<br>
<br>
<center>$Y=(n-1)s^2/\sigma_0^2$</center><br>
<br>
<center>$Y$ ~ $\chi^2(n-1)$이 되는 것을 이용</center>

In [17]:
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'p값은 {p:.3f}')

In [18]:
pvar_test(sample, 9)

귀무가설을 채택
p값은 0.085


### 11.2.3 정규분포의 모평균에 대한 검정 : 모분산을 모르는 경우

- $t$ 검정통계량<br>
<br>
<center>$t=(\bar{X}-\mu_0)/\sqrt{s^2/n}$</center>

In [19]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)

    t = (s_mean - mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p값은 {p:.3f}')

In [20]:
pmean_test(sample, 130)

귀무가설을 채택
p값은 0.169


In [21]:
t, p = stats.ttest_1samp(sample, 130)

t, p

(-1.4551960206404198, 0.16933464230414275)

$t$ 검정통계량과 $p$값 반환

## 11.3 2표본 문제에 관한 가설검정

### 11.3.1 대응비교 $t$ 검정

***대응비교 $t$ 검정(paired t-test)**<br>
: 대응하는 데이터가 있고, 데이터 차이에 정규분포를 가정할 수 있는 경우의 <span style="color:blue">**평균값의 차이**</span>에 대한 검정

In [22]:
training_rel = pd.read_csv('../../../Source/누구나 파이썬 통계분석/source/python_stat_sample-master/data/ch11_training_rel.csv')
print(training_rel.shape)

training_rel.head()

(20, 2)


,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


In [23]:
training_rel['차'] = training_rel['후'] - training_rel['전']

training_rel.head()

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


In [24]:
t, p = stats.ttest_1samp(training_rel['차'], 0)

p

0.04004419061842953

$p$값이 유의수준보다 작기 때문에 귀무가설 기각

In [25]:
t, p = stats.ttest_rel(training_rel['후'], training_rel['전'])

p

0.04004419061842953

### 11.3.2 독립비교 $t$ 검정

***독립비교 $t$ 검정(independend t-test)**<br>
: 대응하는 데이터가 없고 독립된 2표본 모집단에 정규분포를 가정할 수 있는 경우 <span style="color:blue">**평균값의 차이**</span>에 대한 검정

In [26]:
training_ind = pd.read_csv('../../../Source/누구나 파이썬 통계분석/source/python_stat_sample-master/data/ch11_training_ind.csv')
print(training_ind.shape)

training_ind.head()

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [27]:
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'],
                       equal_var=False)

p

0.08695731107259362

$p$값이 유의수준보다 크기 때문에 귀무가설 채택

### 11.3.3 윌콕슨의 부호순위검정

***윌콕슨의 부호순위검정(Wilcoxon signed-rank test)**<br>
: 대응표본에서 차이에 정규분포를 가정할 수 없는 경우, <span style="color:blue">**중앙값의 차이**</span>에 대한 검정

In [28]:
training_rel = pd.read_csv('../../../Source/누구나 파이썬 통계분석/source/python_stat_sample-master/data/ch11_training_rel.csv')
toy_df = training_rel[:6].copy()

toy_df

,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84
5,45,37


In [29]:
diff = toy_df['후'] - toy_df['전']
toy_df['차'] = diff

toy_df

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [30]:
# 차이의 절댓값이 작은 것부터 순서대로 순위를 부여
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['순위'] = rank

toy_df

,전,후,차,순위
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [31]:
# 차이의 부호가 마이너스의 것의 순위합과, 플러스인 것의 순위합
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(8, 13)

둘 중 작은 쪽이 검정통계량 → 8

**윌콕슨의 부호순위검정**에서는<br>
이 <span style="color:green">**검정통계량이 임곗값보다 작은 경우에 귀무가설이 기각되는 단측검정**</span>을 수행

In [32]:
# 극단적인 예 → 근력운동 후의 테스트 결과가 전원 향상된 상황
toy_df['후'] = toy_df['전'] + np.arange(1, 7)
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank

toy_df

,전,후,차,순위
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [33]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(0, 21)

In [34]:
# 근력운동 후의 테스트 결과가 올라간 사람도 있고 내려간 사람도 있는 상황
toy_df['후'] = toy_df['전'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank

toy_df

,전,후,차,순위
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6


In [35]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(11, 10)

- **wilcoxon 함수**로 계산

In [36]:
T, p = stats.wilcoxon(training_rel['전'], training_rel['후'])

p

0.03623390197753906

In [37]:
T, p = stats.wilcoxon(training_rel['후'] - training_rel['전'])

p

0.039989471435546875

$p$값이 유의수준보다 작기 때문에 귀무가설 기각

In [38]:
# 모집단이 정규분포를 따르는 경우
n = 10000
diffs = np.round(stats.norm(3, 4).rvs(size=(n, 20)))

In [39]:
# 대응표본 t 검정
cnt = 0
alpha = 0.05
for diff in diffs:
    t, p = stats.ttest_1samp(diff, 0)
    if p < alpha:
        cnt += 1
cnt / n

0.883

In [40]:
# 윌콕슨의 부호순위검정
cnt = 0
alpha = 0.05
for diff in diffs:
    T, p = stats.wilcoxon(diff)
    if p < alpha:
        cnt += 1
cnt / n

C:\Users\khl06\anaconda3\envs\multi\lib\site-packages\scipy\stats\morestats.py:2967: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


0.873

**모집단이 정규분포를 따르고 있는 경우**에는, 대응비교 $t$ 검정 쪽의 검정력이 크다.

### 11.3.4 만·위트니의 $U$ 검정

***만·위트니의 $U$ 검정(Mann-Whitney rank test)**<br>
: 대응되는 데이터가 없는 2표본 모집단에 정규분포를 가정할 수 없는 경우, <span style="color:blue">**중앙값의 차이**</span>에 대한 검정

In [41]:
training_ind = pd.read_csv('../../../Source/누구나 파이썬 통계분석/source/python_stat_sample-master/data/ch11_training_ind.csv')
toy_df = training_ind[:5].copy()

toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [42]:
# 데이터 전체에 대해서 값이 작은 순서대로 순위를 부여
rank = stats.rankdata(np.concatenate([toy_df['A'],
                                      toy_df['B']]))
rank_df = pd.DataFrame({'A': rank[:5],
                        'B': rank[5:10]}).astype(int)

rank_df

,A,B
0,3,5
1,6,8
2,1,9
3,10,4
4,2,7


A(또는 B)의 순위합에서 A의 크기를 $n_1$로 해서 $n_1(n_1+1)/2$를 뺀 것이 검정통계량<br>
→ 22 - 15 = 7

In [43]:
n1 = len(rank_df['A'])
u = rank_df['A'].sum() - (n1*(n1+1))/2

u

7.0

In [44]:
# A에 좋은 순위가 모두 모여 있는 경우
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5).T,
                       columns=['A', 'B'])

rank_df

,A,B
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [45]:
u = rank_df['A'].sum() - (n1*(n1+1))/2

u

0.0

In [46]:
# A에 좋은 순위가 모두 모여 있는 경우
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5)[::-1].T,
                       columns=['A', 'B'])

rank_df

,A,B
0,6,1
1,7,2
2,8,3
3,9,4
4,10,5


In [47]:
u = rank_df['A'].sum() - (n1*(n1+1))/2

u

25.0

- **mannwhitneyu 함수**로 계산

In [48]:
u, p = stats.mannwhitneyu(training_ind['A'], training_ind['B'],
                          alternative='two-sided')

p

0.05948611166127324

p값이 유의수준보다 크기 때문에 귀무가설 채택

**모집단이 정규분포를 따르고 있는 경우**에는, 독립비교 $t$ 검정 쪽의 검정력이 크다.

### 11.3.5 카이제곱검정

***카이제곱검정(chi-square test) or 독립성 검정(test for independence)**<br>
: 두 변수 $X$와 $Y$에 관해서 '$X$와 $Y$가 독립이다.'라는 귀무가설과 '$X$와 $Y$가 독립이 아니다.'라는 대립가설에 의해 수행되는 검정

In [49]:
ad_df = pd.read_csv('../../../Source/누구나 파이썬 통계분석/source/python_stat_sample-master/data/ch11_ad.csv')
n = len(ad_df)
print(n)

ad_df.head()

1000


,광고,구입
0,B,하지 않았다
1,B,하지 않았다
2,A,했다
3,A,했다
4,B,하지 않았다


In [50]:
# 교차집계표(cross table, 분할표) 작성
ad_cross = pd.crosstab(ad_df['광고'], ad_df['구입'])

ad_cross

구입,하지 않았다,했다
광고,,
A,351,49
B,549,51


In [51]:
ad_cross['했다'] / (ad_cross['했다'] + ad_cross['하지 않았다'])

광고
A    0.1225
B    0.0850
dtype: float64

In [52]:
n_not, n_yes = ad_cross.sum()

n_not, n_yes

(900, 100)

In [53]:
n_adA, n_adB = ad_cross.sum(axis=1)

n_adA, n_adB

(400, 600)

***기대도수(expected frequency)**<br>
: 광고와 구입이 독립인 변수일 때 기대되는 도수<br>
<br>
***관측도수(observed frequency)**<br>
: 실제로 관측된 데이터

In [54]:
# 기대도수 계산
ad_ef = pd.DataFrame({'했다': [n_adA * n_yes / n,
                              n_adB * n_yes / n],
                      '하지 않았다': [n_adA * n_not / n,
                                   n_adB * n_not / n]},
                      index=['A', 'B'])

ad_ef

,했다,하지 않았다
A,40.0,360.0
B,60.0,540.0


- 검정통계량<br>
<br>
<center>$Y=\sum_i\sum_j((O_{ij}-E_{ij})^2/E_{ij})$</center>

In [55]:
y = ((ad_cross - ad_ef) ** 2 / ad_ef).sum().sum()

y

3.75

In [56]:
# p값
rv = stats.chi2(1)
1 - rv.cdf(y)

0.052807511416113395

$p$값이 유의수준보다 크기 때문에 귀무가설 채택<br>
→ 광고 A와 광고 B에 유의한 차이가 인정되지 않는다.

In [57]:
chi2, p, dof, ef = stats.chi2_contingency(ad_cross,
                                          correction=False)

chi2, p, dof

(3.75, 0.052807511416113395, 1)

In [58]:
ef

array([[360.,  40.],
       [540.,  60.]])